# T2 - Text Analysis using Natural Language Processing 

## Problem
The problem selected was the first option: "Problema 1 - Determinação de Valência em Manchetes de Jornais Brasileiros no 1° Semestre de 2017".

### Dataset
The datase used in this experiment is avaliable in [Manchetes de Jornais Brasileiros](https://github.com/pdpcosta/manchetesBrasildatabase)
The dataset 

## Approach

## Experiment Implementation

In [176]:
import nltk
#nltk.download()
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

### Preprocessing

The following preprocessing steps were done:


In [69]:
datasetNews = open("manchetesBrasildatabase.csv", mode="r", encoding="utf-8")
datasetNewsPreprocessed = open("manchetesBrasildatabaseProcessada.csv", mode="w", encoding="utf-8")

for row in datasetNews:
     
    tupleNews = str(row)
    
    tupleNews = tupleNews.replace(",","|")
    
    separatedRow = tupleNews.split("|")

    if len(separatedRow) > 5:
        titleNews = ""
        count = 4
        while count < len(separatedRow):
            titleNews += separatedRow[count]
            if (count < len(separatedRow)-1):
                titleNews += ", "
            count+=1
        separatedRow[4] = titleNews
        
    tuplePreprocessed = separatedRow[0] + " | " + separatedRow[1] + " | " + separatedRow[2] \
                        + " | " + separatedRow[3] + " | " + separatedRow[4]
    
    datasetNewsPreprocessed.write(tuplePreprocessed)

datasetNews.close()
datasetNewsPreprocessed.close()

In [70]:
datasetInFrame = pd.read_csv("manchetesBrasildatabaseProcessada.csv",header=None, sep="|")

In [75]:
datasetInFrame.columns =['Dia', 'Mês', 'Ano', 'Jornal', 'Título']

In [20]:
datasetInFrame.describe()

,Dia,Ano
count,500.000000,500.000000
mean,10.568000,2016.862000
std,8.974322,0.345245
min,1.000000,2016.000000
25%,2.000000,2017.000000
50%,14.000000,2017.000000
75%,17.000000,2017.000000
max,31.000000,2017.000000


In [179]:
sentences = [ str(sentence).lower().replace("'","").replace(".","").replace(",","") for sentence in datasetInFrame["Título"]]

#### Tokenizing

In [161]:
#portuguese_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')

sentencesWithTokens = [nltk.word_tokenize(sentence.lower()) for sentence in sentences]

#### Stop Words

In [165]:
stop_words = stopwords.words('portuguese')

sentencesWithoutStopWords = [word for word in sentencesWithTokens if word not in stop_words]

sentencesWithoutStopWords = []

#Navegando no conjunto de dados
for row in sentencesWithTokens:
    sentence = []
    for word in row:
        if not word in stop_words:
            sentence.append(word)
    #print(sentence)
    sentencesWithoutStopWords.append(sentence)

In [167]:
#sentencesWithoutStopWords

In [93]:
#tags = nltk.pos_tag(sentencesWithTokens[1])

[('bc', 'NN'),
 ('cria', 'NNS'),
 ('novo', 'VBP'),
 ('instrumento', 'NN'),
 ('de', 'FW'),
 ('política', 'FW'),
 ('monetária', 'NNS'),
 ('.', '.')]

In [94]:
nltk.corpus.mac_morpho.words()

['Jersei', 'atinge', 'média', 'de', 'Cr$', '1,4', ...]

### Stop words

['outro tete']

In [136]:
nltk.corpus.mac_morpho.words()

tsents = nltk.corpus.mac_morpho.tagged_sents()
train = tsents[100:]
test = tsents[:100]


## Classification

In [169]:
dataset = pd.read_csv('Tweets_Mg.csv',encoding='utf-8')

In [170]:
dataset[dataset.Classificacao=='Neutro'].count()

Unnamed: 0                   2453
Created At                   2453
Text                         2453
Geo Coordinates.latitude      102
Geo Coordinates.longitude     102
User Location                1712
Username                     2453
User Screen Name             2453
Retweet Count                2453
Classificacao                2453
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [171]:
dataset[dataset.Classificacao=='Positivo'].count()

Unnamed: 0                   3300
Created At                   3300
Text                         3300
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                2118
Username                     3300
User Screen Name             3300
Retweet Count                3300
Classificacao                3300
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [172]:
dataset[dataset.Classificacao=='Negativo'].count()

Unnamed: 0                   2446
Created At                   2446
Text                         2446
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                1659
Username                     2446
User Screen Name             2446
Retweet Count                2446
Classificacao                2446
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [173]:
tweets = dataset['Text'].values
classes = dataset['Classificacao'].values

In [177]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweets)
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [184]:
sentences

' câmbio gera bate-boca entre ua e ue'

In [181]:
freq_testes = vectorizer.transform(sentences)

In [190]:
prediction = modelo.predict(freq_testes)

In [193]:
for i in range(500):
    print("'" + sentences[i] + "' - valence: " + prediction[i])

' bndes encolhe e volta ao nível de 20 anos atrás' - valence: Neutro
' bc cria novo instrumento de política monetária' - valence: Neutro
' câmbio gera bate-boca entre ua e ue' - valence: Neutro
' indenização a transmissoras de energia já chega à tarifa' - valence: Neutro
' políticos esperam que relator separe "joio do trigo"' - valence: Neutro
' philips quer administrar hospitais públicos no brasil' - valence: Neutro
' com vendas em queda  c&amp;c muda lojas e troca diretoria' - valence: Neutro
' fachin poderá ir para turma que julga lava-jato' - valence: Neutro
' eike tem multas que superam fundo para prisões' - valence: Positivo
' operador pagou decoração de luxo de imóveis de cabral' - valence: Neutro
' drama do desemprego está longe de diminuir' - valence: Negativo
' bc prevê queda da inflação a 3%' - valence: Positivo
' boicote atrasa governo trump' - valence: Negativo
' crivella negocia vila dos atletas' - valence: Positivo
' crise amplia diferença de salário entre clt e servidor

In [202]:
datasetInFrame['Valência'] = pd.Series(prediction, index=datasetInFrame.index)
datasetInFrame

,Dia,Mês,Ano,Jornal,Título,Valência
0,1,'fevereiro',2017,'Valor','BNDES encolhe e volta ao nível de 20 anos at...,Neutro
1,1,'fevereiro',2017,'Valor','BC cria novo instrumento de política monetár...,Neutro
2,1,'fevereiro',2017,'Valor','Câmbio gera bate-boca entre UA e UE.',Neutro
3,1,'fevereiro',2017,'Valor','Indenização a transmissoras de energia já ch...,Neutro
4,1,'fevereiro',2017,'Valor',"'Políticos esperam que relator separe ""joio d...",Neutro
5,1,'fevereiro',2017,'Valor','Philips quer administrar hospitais públicos ...,Neutro
6,1,'fevereiro',2017,'Valor',"'Com vendas em queda, C&amp;C muda lojas e t...",Neutro
7,1,'fevereiro',2017,'Globo','Fachin poderá ir para turma que julga Lava-J...,Neutro
8,1,'fevereiro',2017,'Globo','Eike tem multas que superam fundo para prisõ...,Positivo
9,1,'fevereiro',2017,'Globo','Operador pagou decoração de luxo de imóveis ...,Neutro


# Discussion

# Conclusion
The model selected 
I spent a lot of time working with NLTK, I used the examples in English and I tried to adapt to Portuguese.


# References